<a href="https://colab.research.google.com/github/Ria-Chaudhuri/Evaluation/blob/main/output/UnitTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install apache_beam
!pip install apache-beam[gcp]

     |████████████████████████████████| 10.1 MB 4.0 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 2.3 MB 10.4 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 151 kB 54.0 MB/s 
     |████████████████████████████████| 255 kB 66.7 MB/s 
     |████████████████████████████████| 508 kB 53.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.9 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=ac79b405f8b96f882f22bc33fe897f2b235c4c932476ef6dcbc237280d4b9247
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
Successfully built dill
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling

     |████████████████████████████████| 180 kB 3.3 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 113 kB 48.6 MB/s 
     |████████████████████████████████| 265 kB 63.5 MB/s 
     |████████████████████████████████| 267 kB 64.1 MB/s 
     |████████████████████████████████| 183 kB 61.2 MB/s 
     |████████████████████████████████| 173 kB 67.4 MB/s 
     |████████████████████████████████| 435 kB 60.3 MB/s 
     |████████████████████████████████| 179 kB 55.4 MB/s 
     |████████████████████████████████| 255 kB 54.0 MB/s 
     |████████████████████████████████| 232 kB 52.5 MB/s 
     |████████████████████████████████| 114 kB 61.6 MB/s 
     |████████████████████████████████| 267 kB 55.3 MB/s 
     |████████████████████████████████| 267 kB 53.5 MB/s 
     |████████████████████████████████| 266 kB 53.7 MB/s 
     |████████████████████████████████| 266 kB 50.6 MB/s 
     |████████████████████████████████| 265 kB 65.1 MB/s 
     |███████████

In [ ]:
#Imported Libraries and dependencies
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from datetime import datetime
import json
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to

#ParDo Transforms 
class TransactionAmountGreaterThan20(beam.DoFn):
  def process(self, element):
     if float(element[3])>20.00:
          yield element

class ConvertToDateWithTransaction(beam.DoFn):
  def process(self, element):
    date=datetime.strptime((element[0]), "%Y-%m-%d %H:%M:%S %Z").date()
    result=[(date.strftime('%Y-%m-%d'),float(element[3]))]
    return result

#Composite Transform for allplying all the transformation
@beam.ptransform_fn
def TransactionsWithAllTransforms(pcoll):
  return (
      pcoll
                    | 'Split the File'>>beam.Map(lambda x:x.split(","))
                    |'Transactions Greater Than 20  ' >> beam.ParDo(TransactionAmountGreaterThan20())
                    | 'Transactions after 2010  ' >> beam.Filter(lambda record: datetime.strptime((record[0]), "%Y-%m-%d %H:%M:%S %Z").year >2010)
                    | 'Convert To Date With Transactions' >> beam.ParDo(ConvertToDateWithTransaction())
                    | 'Sum values per key' >> beam.CombinePerKey(sum)
                    | 'FormatOutput' >> beam.Map(json.dumps)
         )

pipe= beam.Pipeline()

with beam.Pipeline(options=PipelineOptions()) as pipeline:
  lines = (pipeline | 'ReadMyFile' >> beam.io.ReadFromText('gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv',skip_header_lines=True)
                    |  'Apply All the Transforms'>> TransactionsWithAllTransforms()
                    |beam.Map(print)
                    #|  'Write Output' >> beam.io.WriteToText( file_path_prefix='result.json.gz',header='date, total_amount') 
  )

In [ ]:
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to
from apache_beam.dataframe.io import read_csv
import unittest


class CountTest(unittest.TestCase):

  def test_count(self):
    # Create a test pipeline.
    with TestPipeline() as p:

      # Create an input PCollection.
      input = p | 'ReadMyFile' >> beam.io.ReadFromText('SampleData.csv',skip_header_lines=True)
                 

      # Apply the Count transform under test.
      output = input | TransactionsWithAllTransforms()
                      
                          

      # Assert on the results.
      assert_that(
       output,
        equal_to([['2018-08-09', '100.99'], ['2019-01-09', '547534.23']]))   
            
unittest.main(argv=[''], verbosity=2, exit=False)